In [52]:
import re
import pandas as pd
import numpy as np
import requests

#data = urllib.request.urlopen("https://www.gutenberg.org/cache/epub/1934/pg1934.txt")

#data = http.client.HTTPSConnection("https://www.gutenberg.org/cache/epub/1934/pg1934.txt")

data = requests.get("https://www.gutenberg.org/cache/epub/1934/pg1934.txt", verify=False)

/Users/marenrieker/.local/share/virtualenvs/Assignments-GLOq7LzH/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.gutenberg.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [53]:
text = data.text

In [54]:
index_start = text.find("The Voice of the Ancient Bard\r\n\r\n\r\n\r\n\r\n") 
index_start += len("The Voice of the Ancient Bard\r\n\r\n\r\n\r\n\r\n")
#index_end = text.find("*** END OF THE PROJECT GUTENBERG EBOOK SONGS OF INNOCENCE ANF OF EXPERIENCE ***")
index_end = text.find("And wish to lead others, when they should be led.")

In [55]:
#selecting text from index start to end
text = text[index_start:index_end]

In [56]:
list_poems = text.split("\r\n\r\n\r\n\r\n")
#note that the last line of the last poem is missing; there seems to be a problem with splitting before
#"*** END OF THE PROJECT GUTENBERG EBOOK SONGS OF INNOCENCE ANF OF EXPERIENCE ***", but i chose to keep this
#mistake rather than extracting the last line but also extracting many lines of unnecessary legal statements
#fix if time

In [57]:
pattern = re.compile("[A-Z]+\s?[A-Z]{2}")
def poem_data(poem):
    title = ''
    l = []
    stanza = -1
    line_number = -1
    book_title = "SONGS OF INNOCENCE"
    for zeile in poem.split("\r\n"):
        if zeile == "":
            stanza += 1
        if zeile != "":
            line_number = line_number + 1
        if zeile == "SONGS OF EXPERIENCE":
            book_title = "SONGS OF EXPERIENCE"
        if re.match(pattern, zeile) != None:
            title = zeile
        l.append({"poem_title" : title, "stanza_number" : stanza, 
                  "line_number" : line_number, 'book_title': book_title,
                 'line': zeile})
    return pd.DataFrame.from_records(l)
    

In [58]:
poem_l = [poem_data(poem) for poem in list_poems]

In [59]:
df = pd.concat(poem_l)

In [60]:
df = df[(df['poem_title'] != 'songs of innocence') & (df['poem_title'] != 'songs of experience') & (df['line'] != '')] 

In [61]:
#delete poem titles that were assigned from book titles
df = df[df.poem_title != df.line]

#substracting 1 from the stanza_number, because it somehow starts directly from two
df['stanza_number'] = [st_n - 1 for st_n in df.stanza_number]

In [62]:
df

,poem_title,stanza_number,line_number,book_title,line
4,INTRODUCTION,1,1,SONGS OF INNOCENCE,"Piping down the valleys wild,"
5,INTRODUCTION,1,2,SONGS OF INNOCENCE,"Piping songs of pleasant glee,"
6,INTRODUCTION,1,3,SONGS OF INNOCENCE,"On a cloud I saw a child,"
7,INTRODUCTION,1,4,SONGS OF INNOCENCE,And he laughing said to me:
9,INTRODUCTION,2,5,SONGS OF INNOCENCE,‘Pipe a song about a Lamb!’
...,...,...,...,...,...
9,THE VOICE OF THE ANCIENT BARD,1,6,SONGS OF INNOCENCE,Folly is an endless maze;
10,THE VOICE OF THE ANCIENT BARD,1,7,SONGS OF INNOCENCE,Tangled roots perplex her ways;
11,THE VOICE OF THE ANCIENT BARD,1,8,SONGS OF INNOCENCE,How many have fallen there!
12,THE VOICE OF THE ANCIENT BARD,1,9,SONGS OF INNOCENCE,They stumble all night over bones of the dead;
